In [126]:
# Note this is an exercise for NYU Machine Learning & Finance Course offered by Coursera
# The focus of this is exercise is Liner Regression and calculating Beta hat coefficients
import os
import numpy as np

try:
    import matplotlib.pyplot as plt
    %matplotlib inline
except: pass

import pandas as pd

import tensorflow as tf
from tensorflow.python.layers import core as core_layers
try:
    from mpl_toolkits.mplot3d import Axes3D
except: pass

In [127]:
def reset_graph(seed=42):
    """
    Utility function to reset current tensorflow computation graph
    and set the random seed 
    """
    # to make results reproducible across runs
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [128]:
def generate_data(n_points=10000, n_features=3, use_nonlinear=True, 
                    noise_std=0.1, train_test_split = 4):
    """
    Arguments:
    n_points - number of data points to generate
    n_features - a positive integer - number of features
    use_nonlinear - if True, generate non-linear data
    train_test_split - an integer - what portion of data to use for testing
    
    Return:
    X_train, Y_train, X_test, Y_test, n_train, n_features
    """
    
    # Linear data or non-linear data?
    if use_nonlinear:
        weights = np.array([[1.0, 0.5, 0.2],[0.5, 0.3, 0.15]])
    else:
        weights = np.array([1.0, 0.5, 0.2])
        
    bias = np.ones(n_points).reshape((-1,1))
    low = - np.ones((n_points,n_features),'float')
    high = np.ones((n_points,n_features),'float')
    
    X = np.random.uniform(low=low, high=high)
    noise = np.random.normal(size=(n_points, 1))
    noise_std = 0.1
    
    if use_nonlinear:
        Y = (weights[0,0] * bias + np.dot(X, weights[0, :]).reshape((-1,1)) + 
             np.dot(X*X, weights[1, :]).reshape([-1,1]) +
             noise_std * noise)
    else:
        Y = (weights[0] * bias + np.dot(X, weights[:]).reshape((-1,1)) + 
             noise_std * noise)
    
    n_test = int(n_points/train_test_split)
    n_train = n_points - n_test
    
    X_train = X[:n_train,:]
    Y_train = Y[:n_train].reshape((-1,1))

    X_test = X[n_train:,:]
    Y_test = Y[n_train:].reshape((-1,1))
    
    return X_train, Y_train, X_test, Y_test, n_train, n_features

X_train, Y_train, X_test, Y_test, n_train, n_features = generate_data(use_nonlinear=False)
X_train.shape, Y_train.shape

((7500, 3), (7500, 1))

In [129]:
X = np.c_[np.ones(X_train.shape[0]), X_train]
#X = np.c_[ X_train, np.ones(X_train.shape[0])]
X

array([[ 1.        , -0.55555302,  0.47756895,  0.41859018],
       [ 1.        , -0.50615891,  0.8759816 ,  0.15791025],
       [ 1.        ,  0.83423482, -0.89113633, -0.69812078],
       ..., 
       [ 1.        , -0.07199804, -0.26047484,  0.94713899],
       [ 1.        , -0.96814736,  0.52420104, -0.83776189],
       [ 1.        , -0.46006237,  0.36096295, -0.55180372]])

In [130]:

def numpy_lin_regress(X_train, Y_train):
    """
    numpy_lin_regress - Implements linear regression model using numpy module
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
    np.array of size (k+1 by 1) of regression coefficients
    """
    # number of features
    ndim = X_train.shape[1] 
    X = np.c_[np.ones(X_train.shape[0]), X_train] 
    #X = np.c_[X_train, np.ones(X_train.shape[0])] 
    theta_numpy = np.linalg.lstsq(X,Y_train)[0]
    
    return theta_numpy

In [131]:

theta_numpy = numpy_lin_regress(X_train, Y_train)
theta_numpy.squeeze()


array([ 0.99953953,  1.00239293,  0.50159933,  0.20129424])

In [132]:
from sklearn.linear_model import LinearRegression


def sklearn_lin_regress(X_train, Y_train):
    """
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
    np.array of size (k+1 by 1) of regression coefficients
    """
    ndim = X_train.shape[1] 
   
    lm = LinearRegression()
    lm.fit(X_train,Y_train)
    
    theta_sklearn = np.append(lm.intercept_, lm.coef_)
    
    return theta_sklearn

In [133]:

theta_sklearn = sklearn_lin_regress(X_train, Y_train)
theta_sklearn.squeeze()


array([ 0.99953953,  1.00239293,  0.50159933,  0.20129424])

In [134]:
# Linear Regression with Tensorflow
# A gentle introduction to TensorFlow

def tf_lin_regress(X_train, Y_train):
    """
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
    np.array of size (k+1 by 1) of regression coefficients
    """
    ndim = X_train.shape[1] 
    ### START CODE HERE ### (≈ 7-8 lines of code)
    
    X = tf.placeholder(tf.float32, [None, ndim], name = "X")
    Y = tf.placeholder(tf.float32, [None, 1], name = "Y")
    
    # Regression parameter using the normal equation
    theta_in = tf.placeholder(tf.float32, [ndim + 1, None])
    
    data_plus_bias = tf.concat([tf.ones([tf.shape(X)[0], 1]), X], axis =1)

    XT = tf.transpose(data_plus_bias)
    
    theta_value = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, data_plus_bias)), XT), Y)
    
    # Start a session to be able to run operations
    
    with tf.Session() as sess: 
        result = sess.run(theta_value,feed_dict={X: X_train, Y: Y_train})
        #print(result)
    sess.close()
    
    #print(result)
    
    return result


In [135]:

print("Linear Regression Beta_hat using TensorFlow:")
print()
theta_tf = tf_lin_regress(X_train, Y_train)
theta_tf.squeeze()


Linear Regression Beta_hat using TensorFlow:



array([ 0.99953949,  1.00239229,  0.50159907,  0.20129436], dtype=float32)

In [136]:
print("Linear Regression Beta_hat using SciKit Learn:")
print()
theta_sklearn = sklearn_lin_regress(X_train, Y_train)
theta_sklearn.squeeze()

Linear Regression Beta_hat using SciKit Learn:



array([ 0.99953953,  1.00239293,  0.50159933,  0.20129424])

In [137]:
print("Linear Regression Beta_hat using NumPy:")
print()
theta_numpy = numpy_lin_regress(X_train, Y_train)
theta_numpy.squeeze()


Linear Regression Beta_hat using NumPy:



array([ 0.99953953,  1.00239293,  0.50159933,  0.20129424])

In [138]:

def run_normal_eq(X_train, Y_train, X_test, Y_test, learning_rate=0.05):
    """
    Implements normal equation using tensorflow, trains the model using training data set
    Tests the model quality by computing mean square error (MSE) of the test data set
    
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    X_test  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_test - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    
    Return a tuple of:
        - np.array of size (k+1 by 1) of regression coefficients
        - mean square error (MSE) of the test data set
        - mean square error (MSE) of the training data set
    """
    ndim_train = X_train.shape[1]
    ndim_test = X_test.shape[1]
    
    X_tr = tf.placeholder(tf.float32, [None, ndim_train], name = "X_tr")
    Y_tr = tf.placeholder(tf.float32, [None, 1], name = "Y_tr")
    
    X_tst = tf.placeholder(tf.float32, [None, ndim_test], name = "X_tst")
    Y_tst = tf.placeholder(tf.float32, [None, 1], name = "Y_tst")
    
    data_plus_bias_train = tf.concat([tf.ones([tf.shape(X_tr)[0], 1]), X_tr], axis =1)
    data_plus_bias_test = tf.concat([tf.ones([tf.shape(X_tst)[0], 1]), X_tst], axis =1)
    
    lr_mse_train = 0.
    lr_mse_test = 0.
    
    
    XT = tf.transpose(data_plus_bias_train)
    
    theta_value = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, data_plus_bias_train)), XT), Y_tr)
    
    lr_mse_train = tf.reduce_mean(tf.square(tf.matmul(data_plus_bias_train, theta_value) - Y_tr))
    lr_mse_test = tf.reduce_mean(tf.square(tf.matmul(data_plus_bias_test, theta_value) - Y_tst))
    
    # Start a session to be able to run operations
    
    with tf.Session() as sess: 
        result, lr_mse_train, lr_mse_test = sess.run([theta_value,lr_mse_train,lr_mse_test],\
                                                          feed_dict={X_tr: X_train, Y_tr: Y_train, X_tst: X_test, Y_tst: Y_test})
        print("Linear Regression Mean Squared Error for Training Data:")
        print(lr_mse_train)
        print()
        print("Linear Regression Mean Squared Error for Test Data:")
        print(lr_mse_test)
        print()
        
    sess.close()
    
   
    return result, lr_mse_train, lr_mse_test



In [139]:
theta_value, lr_mse_train, lr_mse_test = run_normal_eq(X_train, Y_train, X_test, Y_test)

Linear Regression Mean Squared Error for Training Data:
0.00977294

Linear Regression Mean Squared Error for Test Data:
0.0103219



In [140]:

theta_value.squeeze()


array([ 0.99953949,  1.00239229,  0.50159907,  0.20129436], dtype=float32)

In [141]:
print("Linear Regression Beta_hat using NumPy:")
print()
theta_numpy = numpy_lin_regress(X_train, Y_train)
theta_numpy.squeeze()

Linear Regression Beta_hat using NumPy:



array([ 0.99953953,  1.00239293,  0.50159933,  0.20129424])

In [142]:
lm = LinearRegression()
lm.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [143]:
# Make predictions using the testing set
Y_Train_pred = lm.predict(X_train)
Y_Test_pred = lm.predict(X_test)


In [144]:
from sklearn.metrics import mean_squared_error

mse_train = mean_squared_error(Y_train, Y_Train_pred)
mse_test = mean_squared_error(Y_test, Y_Test_pred)

In [145]:
print("Using SciKit Learn:")
print("Linear Regression Mean Square Error for Training Data:")
print(mse_train)
print()
print("Linear Regression Mean Square Error for Test Data:")
print(mse_test)
print()

Using SciKit Learn:
Linear Regression Mean Square Error for Training Data:
0.00977294340974

Linear Regression Mean Square Error for Test Data:
0.0103219324219



In [146]:
print("Using TensorFlow:")
theta_value, lr_mse_train, lr_mse_test = run_normal_eq(X_train, Y_train, X_test, Y_test)

Using TensorFlow:
Linear Regression Mean Squared Error for Training Data:
0.00977294

Linear Regression Mean Squared Error for Test Data:
0.0103219



In [147]:

def numpy_lin_reg(X_train, Y_train):
    """
    numpy_lin_regress - Implements linear regression model using numpy module
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
    np.array of size (k+1 by 1) of regression coefficients
    """
    # Note: This function calculates the Beta hat coefficients by explictiy using the
    # normal equation: w(Beta hat) =(XTX)−1*XT*Y
    # number of features
    ndim = X_train.shape[1] 
    X = np.c_[np.ones(X_train.shape[0]), X_train]
    X_T = np.transpose(X)
    ralph = np.linalg.inv(np.dot(X_T,X))
    selina = np.dot(X_T,Y_train)
    # theta_numpy is Beta hat coefficients
    theta_numpy = np.dot(ralph,selina)

    return theta_numpy

In [148]:
print("Linear Regression Beta_hat using NumPy:")
print()
theta_numpy = numpy_lin_regress(X_train, Y_train)
theta_numpy.squeeze()

Linear Regression Beta_hat using NumPy:



array([ 0.99953953,  1.00239293,  0.50159933,  0.20129424])

In [149]:
print("Linear Regression Beta_hat using NumPy version 2:")
print()
theta_numpy_2 = numpy_lin_reg(X_train, Y_train)
theta_numpy_2.squeeze()

Linear Regression Beta_hat using NumPy version 2:



array([ 0.99953953,  1.00239293,  0.50159933,  0.20129424])

In [150]:
print("Linear Regression Beta_hat using SciKit Learn:")
print()
theta_sklearn = sklearn_lin_regress(X_train, Y_train)
theta_sklearn.squeeze()

Linear Regression Beta_hat using SciKit Learn:



array([ 0.99953953,  1.00239293,  0.50159933,  0.20129424])